In [1]:
import os
import sys
import time

from alerce.core import Alerce
alerce = Alerce()

import pickle
import numpy as np
import pandas as pd
from itertools import chain
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%config InlineBackend.figure_format = 'retina'

In [2]:
# From 106 available features from Kostya's lc_feature_extractor, use the 82 from SNAD Miner paper 
# R and g bands
feature_names_r_g = ['feature_amplitude_magn_r', 
                         'feature_anderson_darling_normal_magn_r',
                         'feature_beyond_1_std_magn_r', 
                         'feature_beyond_2_std_magn_r',
                         'feature_cusum_magn_r', 
                         #'feature_eta_e_magn_r',
                         'feature_inter_percentile_range_2_magn_r',
                         'feature_inter_percentile_range_10_magn_r',
                         'feature_inter_percentile_range_25_magn_r', 
                         'feature_kurtosis_magn_r',
                         'feature_linear_fit_slope_magn_r',
                         'feature_linear_fit_slope_sigma_magn_r',
                         #'feature_linear_fit_reduced_chi2_magn_r', 
                         #'feature_linear_trend_magn_r', # cadence removal
                         #'feature_linear_trend_sigma_magn_r',  # cadence removal
                         'feature_magnitude_percentage_ratio_40_5_magn_r',
                         'feature_magnitude_percentage_ratio_20_5_magn_r',
                         #'feature_maximum_slope_magn_r',
                         'feature_mean_magn_r',
                         'feature_median_absolute_deviation_magn_r',
                         'feature_percent_amplitude_magn_r',
                         'feature_median_buffer_range_percentage_10_magn_r',
                         'feature_median_buffer_range_percentage_20_magn_r',
                         'feature_percent_difference_magnitude_percentile_5_magn_r',
                         'feature_percent_difference_magnitude_percentile_10_magn_r',
                         #'feature_period_0_magn_r',  # should be negated
                         #'feature_period_s_to_n_0_magn_r', # cadence removal
                         #'feature_period_1_magn_r', 
                         #'feature_period_s_to_n_1_magn_r', # cadence removal
                         #'feature_period_2_magn_r', 
                         #'feature_period_s_to_n_2_magn_r', # cadence removal
                         #'feature_period_3_magn_r', 
                         #'feature_period_s_to_n_3_magn_r', # cadence removal
                         #'feature_period_4_magn_r', 
                         #'feature_period_s_to_n_4_magn_r', # cadence removal
                         #'feature_periodogram_amplitude_magn_r',
                         #'feature_periodogram_beyond_2_std_magn_r',  # cadence removal
                         #'feature_periodogram_beyond_3_std_magn_r',  # cadence removal
                         #'feature_periodogram_standard_deviation_magn_r',   # cadence removal
                         #'feature_chi2_magn_r',
                         'feature_skew_magn_r', 
                         'feature_standard_deviation_magn_r',
                         'feature_stetson_k_magn_r',
                         'feature_weighted_mean_magn_r',
                         'feature_anderson_darling_normal_flux_r', 
                         'feature_cusum_flux_r',
                         #'feature_eta_e_flux_r', 
                         'feature_excess_variance_flux_r',
                         'feature_kurtosis_flux_r', 
                         'feature_mean_variance_flux_r',
                         #'feature_chi2_flux_r', 
                         'feature_skew_flux_r',
                         'feature_stetson_k_flux_r',

                         'feature_amplitude_magn_g', 
                         'feature_anderson_darling_normal_magn_g',
                         'feature_beyond_1_std_magn_g', 
                         'feature_beyond_2_std_magn_g',
                         'feature_cusum_magn_g', 
                         #'feature_eta_e_magn_g',
                         'feature_inter_percentile_range_2_magn_g',
                         'feature_inter_percentile_range_10_magn_g',
                         'feature_inter_percentile_range_25_magn_g', 
                         'feature_kurtosis_magn_g',
                         'feature_linear_fit_slope_magn_g',
                         'feature_linear_fit_slope_sigma_magn_g',
                         #'feature_linear_fit_reduced_chi2_magn_g', 
                         #'feature_linear_trend_magn_g', # cadence removal
                         #'feature_linear_trend_sigma_magn_g',  # cadence removal
                         'feature_magnitude_percentage_ratio_40_5_magn_g',
                         'feature_magnitude_percentage_ratio_20_5_magn_g',
                         #'feature_maximum_slope_magn_g', 
                         'feature_mean_magn_g',
                         'feature_median_absolute_deviation_magn_g',
                         'feature_median_buffer_range_percentage_10_magn_g',
                         'feature_median_buffer_range_percentage_20_magn_g',
                         'feature_percent_amplitude_magn_g',
                         'feature_percent_difference_magnitude_percentile_5_magn_g',
                         'feature_percent_difference_magnitude_percentile_10_magn_g',
                         #'feature_period_0_magn_g',  # should be negated
                         #'feature_period_s_to_n_0_magn_g', # cadence removal
                         #'feature_period_1_magn_g', 
                         #'feature_period_s_to_n_1_magn_g', # cadence removal
                         #'feature_period_2_magn_g', 
                         #'feature_period_s_to_n_2_magn_g', # cadence removal
                         #'feature_period_3_magn_g', 
                         #'feature_period_s_to_n_3_magn_g', # cadence removal
                         #'feature_period_4_magn_g', 
                         #'feature_period_s_to_n_4_magn_g', # cadence removal
                         #'feature_periodogram_amplitude_magn_g',
                         #'feature_periodogram_beyond_2_std_magn_g',  # cadence removal
                         #'feature_periodogram_beyond_3_std_magn_g', # cadence removal
                         #'feature_periodogram_standard_deviation_magn_g',  # cadence removal
                         #'feature_chi2_magn_g',
                         'feature_skew_magn_g', 
                         'feature_standard_deviation_magn_g',
                         'feature_stetson_k_magn_g', 
                         'feature_weighted_mean_magn_g',
                         'feature_anderson_darling_normal_flux_g', 
                         'feature_cusum_flux_g',
                         #'feature_eta_e_flux_g', 
                         'feature_excess_variance_flux_g',
                         'feature_kurtosis_flux_g', 
                         'feature_mean_variance_flux_g',
                         #'feature_chi2_flux_g', 
                         'feature_skew_flux_g',
                         'feature_stetson_k_flux_g'] 

In [3]:
def get_alerce_class(output):
    top_class = None
    for item in output:
        if item['ranking'] == 1:
            if top_class is None or \
               (item['classifier_name'] == 'lc_classifier' and \
                item['classifier_version'] == 'hierarchical_rf_1.1.0') or \
               (item['classifier_name'] == 'stamp_classifier' and \
                item['classifier_version'] == 'stamp_classifier_1.0.4' and \
                top_class['classifier_version'] != 'hierarchical_rf_1.1.0') or \
               (item['classifier_name'] == 'stamp_classifier' and \
                item['classifier_version'] == 'stamp_classifier_1.0.0' and \
                top_class['classifier_version'] not in ['hierarchical_rf_1.1.0', 'stamp_classifier_1.0.4']):
                top_class = item
    return top_class['class_name'] if top_class is not None else "Not_classified"

# Load raw dataset

In [4]:
%%time

# Grab raw 4GB loci dataset from Adam
with open("../loci_dbs/loci.pkl", "rb") as f:
    loci = pickle.load(f)

CPU times: user 48.2 s, sys: 1min 33s, total: 2min 21s
Wall time: 3min 17s


In [5]:
len(loci)

2536582

# Split into chunks for easier processing

In [14]:
%%time

loci_df = pd.DataFrame.from_dict(loci[2000000:])
print("made DF")
loci_df = pd.concat([loci_df.drop(['properties'], axis=1), loci_df['properties'].apply(pd.Series)], axis=1)

coord_testset = SkyCoord(list(loci_df['ra']), 
                         list(loci_df['dec']), 
                         frame="icrs", 
                         unit='deg')
galcoord_testset = coord_testset.galactic
loci_df['locus_gal_l'] = galcoord_testset.l.degree
loci_df['locus_gal_b'] = galcoord_testset.b.degree
loci_df.drop('replaced_by', axis=1, inplace=True)
loci_df.drop('htm16', axis=1, inplace=True)
loci_df.drop('watch_list_ids', axis=1, inplace=True)
loci_df.drop('watch_object_ids', axis=1, inplace=True)
print("begin cuts")

# cut on anything in galactic plane
hard_mask = (np.abs(loci_df['locus_gal_b']) < 15) 
loci_df = loci_df[~hard_mask]

# Don't cut on catalogs, because many transients are actually in galaxies whose loci are in them
# # cut on anything that's a likely star (in a star catalog), or AGN
# # 'bright_guide_star_cat' is sometimes a galaxy, so keep
# drop_values = ['sdss_stars', 'asassn_variable_catalog', 'asassn_variable_catalog_v2_20190802', 'veron_agn_qso']
# loci_df = loci_df[~loci_df['catalogs'].apply(lambda x: any(item in x for item in drop_values))]

loci_df.to_csv(f'../loci_dbs/loci_df_{len(loci_df)}objects_2000000+_cut_gal_plane_NOtransf_041223.csv.gz', compression='gzip')

made DF
begin cuts
CPU times: user 4min 55s, sys: 3min 18s, total: 8min 13s
Wall time: 10min 24s


In [15]:
loci_df.head()

,locus_id,ra,dec,tags,catalogs,ztf_object_id,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,anomaly_score,anomaly_mask,vpdf_extreme_version,vpdf_extreme_bright,anomaly,anom_score,vpdf_extreme_faint,horizons_targetname,locus_gal_l,locus_gal_b
1,ANT2020hnqhw,158.579787,-5.424217,"[lc_feature_extractor, matheson_extreme_vpdf, ...","[gaia_dr2, allwise, bright_guide_star_cat]",ZTF20aacxmgz,null,190,114,ztf_candidate:1098438320415015015,...,0.438297,Fail,0.1.0,pct_99.9999_outlier,NaN,NaN,NaN,NaN,252.249323,43.550890
2,ANT2020pwlra,66.916468,-24.112739,[lc_feature_extractor],"[2mass_psc, gaia_dr2, allwise, bright_guide_st...",ZTF19abzevhg,null,36,6,ztf_candidate:988522444315010035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.102139,-41.537241
3,ANT2020edsjc,147.450669,-23.811031,[lc_feature_extractor],"[2mass_psc, gaia_dr2, allwise, bright_guide_st...",ZTF19aarqlkc,null,135,34,ztf_candidate:844142624215010011,...,0.377146,Fail,NaN,NaN,NaN,NaN,NaN,NaN,257.960984,22.801351
4,ANT2020swsp6,341.188090,16.051393,[lc_feature_extractor],"[2mass_psc, bright_guide_star_cat, gaia_dr2, a...",ZTF19abuarlx,null,149,18,ztf_candidate:980394230715015000,...,0.428439,Pass,NaN,NaN,NaN,NaN,NaN,NaN,83.748003,-37.001519
5,ANT2020rzau4,139.405775,-13.330009,[lc_feature_extractor],"[bright_guide_star_cat, 2mass_psc, gaia_dr2, a...",ZTF19aaqnlgj,null,34,7,ztf_candidate:1118314384815015029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,243.915426,24.239616


In [16]:
loci_df.head()['feature_percent_amplitude_magn_g']

1    0.701191
2    0.645111
3    0.548532
4    0.411844
5         NaN
Name: feature_percent_amplitude_magn_g, dtype: float64

# Merge DFs created above

In [4]:
# Merge DFs and then cut on ztf g,r features

loci_df_1 = pd.read_csv(f'../loci_dbs/loci_df_417846objects_0-1000000_cut_gal_plane_NOtransf_041223.csv.gz', compression='gzip')
loci_df_2 = pd.read_csv(f'../loci_dbs/loci_df_359552objects_1000000-2000000_cut_gal_plane_NOtransf_041223.csv.gz', compression='gzip')
loci_df_3 = pd.read_csv(f'../loci_dbs/loci_df_199444objects_2000000+_cut_gal_plane_NOtransf_041223.csv.gz', compression='gzip')

merge_df = pd.concat([loci_df_1, loci_df_2, loci_df_3], axis=0)
merge_df

,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_object_id,ztf_ssnamenr,num_alerts,num_mag_values,...,anom_score,anomaly_score,anomaly_mask,anomaly_type,is_corrected,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b
0,0,ANT2019thdpe,258.233521,17.448168,['lc_feature_extractor'],"['allwise', 'gaia_dr2', 'bright_guide_star_cat']",ZTF19aascxuy,NaN,34,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.568709,29.374214
1,3,ANT2019fwynm,56.003468,-27.191885,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",ZTF21aaaazgo,NaN,101,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223.252671,-51.810753
2,4,ANT2020oxpnu,314.873962,10.929879,['lc_feature_extractor'],"['2mass_psc', 'allwise', 'gaia_dr2', 'bright_g...",ZTF18acmaxoi,NaN,52,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.934277,-22.135358
3,5,ANT2020aer3s3i,34.977270,-19.964847,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",ZTF20acilkri,NaN,62,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198.183274,-68.534464
4,6,ANT2020rajqu,33.976088,-20.258905,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",ZTF20acpdyyq,NaN,137,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197.881532,-69.513571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199439,536574,ANT2019mb736,309.320054,15.325669,['lc_feature_extractor'],"['allwise', '2mass_psc', 'sdss_stars', 'bright...",ZTF18ablguqn,NaN,280,16,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,59.469289,-15.185301
199440,536575,ANT2022bruzw,309.246287,15.453591,['lc_feature_extractor'],"['2mass_psc', 'gaia_dr2', 'allwise', 'bright_g...",ZTF22aacuqam,NaN,57,8,...,NaN,NaN,NaN,TBD,True,NaN,NaN,NaN,59.537116,-15.054401
199441,536576,ANT2022bhwbg,309.425958,15.357704,['lc_feature_extractor'],"['2mass_psc', 'gaia_dr2', 'allwise', 'sdss_sta...",ZTF21aaajgyj,NaN,62,7,...,NaN,NaN,NaN,TBD,True,NaN,NaN,NaN,59.556453,-15.251504
199442,536578,ANT2020kygei,309.184771,15.360183,['lc_feature_extractor'],"['allwise', 'sdss_stars', '2mass_psc', 'gaia_d...",ZTF19aacmnqe,NaN,51,5,...,NaN,NaN,NaN,TBD,True,NaN,NaN,NaN,59.422602,-15.058100


# Cut for g,r features are not null

In [5]:
# drop row if any feature values are nan
merge_df_gr = merge_df[~merge_df[feature_names_r_g].isnull().any(axis=1)]
merge_df_gr = merge_df_gr.set_index('ztf_object_id')
#merge_df_gr.to_csv(f'../loci_dbs/merge_df_gr_{len(merge_df_gr)}objects_cut_gal_plane_NOtransf_041223.csv.gz', compression='gzip')
merge_df_gr

,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,anom_score,anomaly_score,anomaly_mask,anomaly_type,is_corrected,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF20acpdyyq,6,ANT2020rajqu,33.976088,-20.258905,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",NaN,137,35,ztf_candidate:993481181415010007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197.881532,-69.513571
ZTF18acfvwwa,39,ANT2020pc6ko,35.639509,32.833982,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",NaN,659,88,ztf_candidate:943483942915015010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.200342,-26.246909
ZTF17aaaskvj,58,ANT2020rdpmm,28.015003,36.953217,['lc_feature_extractor'],"['allwise', 'csdr2', 'gaia_dr2', '2mass_psc', ...",NaN,1497,579,ztf_candidate:598425334815010002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.189482,-24.352953
ZTF17aaaskvj,59,ANT2018ftblk,28.015003,36.953216,"['lc_feature_extractor', 'high_snr']","['allwise', 'csdr2', 'gaia_dr2', '2mass_psc', ...",NaN,1504,583,ztf_candidate:598425334815010002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.189482,-24.352954
ZTF18abpxcqh,61,ANT2018g55uk,22.061385,4.097728,['lc_feature_extractor'],"['asassn_variable_catalog', 'allwise', 'csdr2'...",NaN,38,12,ztf_candidate:611410431615010001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.217662,-57.533478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF18abdjxbt,536540,ANT2018iuivq,318.453753,19.468958,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', '2mass_ps...",NaN,329,181,ztf_candidate:546400373815015007,...,NaN,0.436664,Fail,Inlier,True,NaN,NaN,NaN,68.431914,-19.658538
ZTF18aazvaxh,536548,ANT2020azeko,314.705034,22.348679,"['high_flux_ratio_wrt_nn', 'lc_feature_extract...","['allwise', '2mass_psc', 'bright_guide_star_ca...",NaN,562,383,ztf_candidate:527419576115010003,...,NaN,0.519887,Fail,Inlier,True,NaN,NaN,NaN,68.457050,-15.125217
ZTF21abzflvc,536549,ANT2019ani6y,316.187923,15.437068,['lc_feature_extractor'],"['2mass_psc', 'bright_guide_star_cat', 'gaia_d...",NaN,510,114,ztf_candidate:1057112081015015143,...,NaN,0.414404,Fail,Inlier,True,NaN,NaN,NaN,63.641741,-20.485976


In [6]:
merge_df_gr.loc['ZTF17aaahrni']['feature_anderson_darling_normal_magn_r']

0.581482168069246

# Cut on alerce classifier to keep only transients

In [7]:
%%time
# get alerce stamp_class from keep_df, and then cut to just SN like things, and use that to cut for 
# which dataset_bank_orig to run

merge_df_gr_alerce = merge_df_gr.copy()

start_idx_l = list(np.arange(0, len(merge_df_gr_alerce), 10000)) 
end_idx_l = list(np.arange(10000, len(merge_df_gr_alerce)+10000, 10000))

for start, end in zip(start_idx_l, end_idx_l):
    start_idx, end_idx = start, end
    print("\nSTART:", start, "END:", end)

    merge_df_gr_alerce_mini = merge_df_gr_alerce.iloc[start_idx:end_idx]
    
    stamp_cls_l = []
    for ztfid in merge_df_gr_alerce_mini.index:
        try:
            probabilities = alerce.query_probabilities(ztfid)
            stamp_cls = get_alerce_class(probabilities)
        except:
            stamp_cls = 'Error'

        stamp_cls_l.append(stamp_cls)
        
    merge_df_gr_alerce_mini['stamp_cls'] = stamp_cls_l
    merge_df_gr_alerce_mini.to_csv(f'../loci_dbs/alerce_cut/merge_df_gr_alerce_mini_start_idx-end_idx={start_idx}-{end_idx}.csv.gz', compression='gzip')
    print("saved!")
    print(merge_df_gr_alerce_mini.head())
    print("##############")


START: 0 END: 10000
saved!
               Unnamed: 0      locus_id         ra        dec  \
ztf_object_id                                                   
ZTF20acpdyyq            6  ANT2020rajqu  33.976088 -20.258905   
ZTF18acfvwwa           39  ANT2020pc6ko  35.639509  32.833982   
ZTF17aaaskvj           58  ANT2020rdpmm  28.015003  36.953217   
ZTF17aaaskvj           59  ANT2018ftblk  28.015003  36.953216   
ZTF18abpxcqh           61  ANT2018g55uk  22.061385   4.097728   

                                               tags  \
ztf_object_id                                         
ZTF20acpdyyq               ['lc_feature_extractor']   
ZTF18acfvwwa               ['lc_feature_extractor']   
ZTF17aaaskvj               ['lc_feature_extractor']   
ZTF17aaaskvj   ['lc_feature_extractor', 'high_snr']   
ZTF18abpxcqh               ['lc_feature_extractor']   

                                                        catalogs ztf_ssnamenr  \
ztf_object_id                                    

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF18accnreo       221752  ANT2020fkigs  300.512320  70.993771   
ZTF18aacboll       221753  ANT2020jnf2g  189.136044   8.122750   
ZTF20acbeyyv       221755  ANT202044rc6   28.422180 -11.959802   
ZTF18abbdzxv       221760  ANT2019a6py6  326.180683  27.205135   
ZTF19abbrezu       221773  ANT2020fxym4  295.220608  68.399479   

                                   tags  \
ztf_object_id                             
ZTF18accnreo   ['lc_feature_extractor']   
ZTF18aacboll   ['lc_feature_extractor']   
ZTF20acbeyyv   ['lc_feature_extractor']   
ZTF18abbdzxv   ['lc_feature_extractor']   
ZTF19abbrezu   ['lc_feature_extractor']   

                                                        catalogs ztf_ssnamenr  \
ztf_object_id                                                                   
ZTF18accnreo   ['2mass_psc', 'bright_guide_star_cat', 'allwis...  

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF19aawsbxe       418408  ANT2019am4ia  340.120262  -1.113853   
ZTF18actwqti       418409  ANT2020ca4u4  304.974707  -2.383652   
ZTF19abezrhy       418415  ANT2019ar4xg  357.325745  11.327092   
ZTF19aacxqvy       418416  ANT2020blieq  165.874626   5.860354   
ZTF18acqrylr       418423  ANT2020h6p6o  119.819719  51.135640   

                                                            tags  \
ztf_object_id                                                      
ZTF19aawsbxe           ['lc_feature_extractor', 'extragalactic']   
ZTF18actwqti                 ['dimmers', 'lc_feature_extractor']   
ZTF19abezrhy                            ['lc_feature_extractor']   
ZTF19aacxqvy   ['extragalactic', 'lc_feature_extractor', 'dim...   
ZTF18acqrylr                            ['lc_feature_extractor']   

                                                     

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF18abbfotj       593763  ANT2018bqpnc  337.372400  10.824294   
ZTF19abocpvh       593768  ANT2020rfmgu   66.595031  23.013461   
ZTF18abvzmdo       593769  ANT2020ejycg  346.533836  34.271761   
ZTF18abieasp       593786  ANT2020fjkhi  240.887932  80.013518   
ZTF18aanfupe       593791  ANT2018d3hlo  270.393530  50.813834   

                                   tags  \
ztf_object_id                             
ZTF18abbfotj   ['lc_feature_extractor']   
ZTF19abocpvh   ['lc_feature_extractor']   
ZTF18abvzmdo   ['lc_feature_extractor']   
ZTF18abieasp   ['lc_feature_extractor']   
ZTF18aanfupe   ['lc_feature_extractor']   

                                                        catalogs ztf_ssnamenr  \
ztf_object_id                                                                   
ZTF18abbfotj   ['gaia_dr2', 'bright_guide_star_cat', 'allwise...  

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF18abkeqjd       795032  ANT2018dpa4g  257.008771   3.646191   
ZTF18abpwjyc       795055  ANT2018f6m3u  356.732595  36.884172   
ZTF21abjvwdj       795056  ANT2021si6do    9.889266  12.244862   
ZTF19aaeypyj       795072  ANT2019a4jls  329.767795  27.762671   
ZTF18abshxcw       795078  ANT2020ewms4  236.625703   5.435887   

                                                            tags  \
ztf_object_id                                                      
ZTF18abkeqjd                            ['lc_feature_extractor']   
ZTF18abpwjyc                            ['lc_feature_extractor']   
ZTF21abjvwdj   ['refitt_newsources_snrcut', 'nova_test', 'ext...   
ZTF19aaeypyj                            ['lc_feature_extractor']   
ZTF18abshxcw                 ['lc_feature_extractor', 'dimmers']   

                                                     

...

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF18acgnuap        73646  ANT2020kbiau   96.865947  56.238094   
ZTF20acflgka        73649  ANT20207ecti   91.126759 -14.686677   
ZTF18acaqmgr        73653  ANT2019cfpba  311.803080  -9.934256   
ZTF18acwrstg        73654  ANT2020odjmq  312.473050 -11.494267   
ZTF18actxdnf        73655  ANT2019axff4   20.036655  34.599157   

                                   tags  \
ztf_object_id                             
ZTF18acgnuap   ['lc_feature_extractor']   
ZTF20acflgka   ['lc_feature_extractor']   
ZTF18acaqmgr   ['lc_feature_extractor']   
ZTF18acwrstg   ['lc_feature_extractor']   
ZTF18actxdnf   ['lc_feature_extractor']   

                                                        catalogs ztf_ssnamenr  \
ztf_object_id                                                                   
ZTF18acgnuap   ['bright_guide_star_cat', '2mass_psc', 'gaia_d...  

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF17aaajoow       200172  ANT2019wjvze  189.041845  22.395025   
ZTF19aadunlp       200176  ANT2020bzeh6  245.636550 -20.324953   
ZTF18aaajfrl       200178  ANT2020atuak  191.556140  18.468983   
ZTF18aaaayjd       200199  ANT2020lx6ie  113.724322  19.271002   
ZTF17aaajouo       200201  ANT2020ecuyw  176.739895  14.040597   

                                               tags  \
ztf_object_id                                         
ZTF17aaajoow   ['lc_feature_extractor', 'high_snr']   
ZTF19aadunlp               ['lc_feature_extractor']   
ZTF18aaajfrl   ['high_snr', 'lc_feature_extractor']   
ZTF18aaaayjd    ['dimmers', 'lc_feature_extractor']   
ZTF17aaajouo   ['lc_feature_extractor', 'high_snr']   

                                                        catalogs ztf_ssnamenr  \
ztf_object_id                                                  

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF17aabclkd       317483  ANT2019gu3ra  100.521147  46.575871   
ZTF18abmxabh       317489  ANT2020a4n2w  255.516796   3.648618   
ZTF18aawffzy       317491  ANT2020cfgni  230.370465   5.972831   
ZTF18aabiagp       317499  ANT2018g6qbe  109.604711  66.632545   
ZTF19aalszlg       317504  ANT2020e5p3g  218.385316  -3.717687   

                                                          tags  \
ztf_object_id                                                    
ZTF17aabclkd   ['siena_mag_coord_cut', 'lc_feature_extractor']   
ZTF18abmxabh                          ['lc_feature_extractor']   
ZTF18aawffzy              ['lc_feature_extractor', 'high_snr']   
ZTF18aabiagp                          ['lc_feature_extractor']   
ZTF19aalszlg               ['lc_feature_extractor', 'dimmers']   

                                                        catalogs zt

saved!
               Unnamed: 0        locus_id          ra        dec  \
ztf_object_id                                                      
ZTF18aaacezr       404636    ANT2020bmymc  149.071703  62.878582   
ZTF18acgvwck       404638  ANT2020aeswazq  153.692992  59.490092   
ZTF18abimnvo       404642    ANT2020cs2ng  271.010107  24.464642   
ZTF19aaeslyh       404643    ANT2020d5hrc  246.966813 -17.896312   
ZTF18aaacdmc       404644    ANT2020bjsa4  136.201710  47.387709   

                                                          tags  \
ztf_object_id                                                    
ZTF18aaacezr   ['lc_feature_extractor', 'siena_mag_coord_cut']   
ZTF18acgvwck                          ['lc_feature_extractor']   
ZTF18abimnvo                          ['lc_feature_extractor']   
ZTF19aaeslyh              ['high_snr', 'lc_feature_extractor']   
ZTF18aaacdmc         ['lc_feature_extractor', 'extragalactic']   

                                                     

saved!
               Unnamed: 0      locus_id          ra        dec  \
ztf_object_id                                                    
ZTF18adbaisv       510463  ANT20193jqoi  230.291364 -19.494715   
ZTF18acuswew       510466  ANT2020fckkq  219.022920  -1.399991   
ZTF20aatvdvr       510467  ANT2022ahaw4  216.188532   4.229655   
ZTF18acyygbu       510469  ANT2020bdauc  225.803047 -12.742080   
ZTF19aaapkyv       510472  ANT2020atjca  219.174794   2.494618   

                                                            tags  \
ztf_object_id                                                      
ZTF18adbaisv   ['young_extragalactic_candidate', 'lc_feature_...   
ZTF18acuswew                ['high_snr', 'lc_feature_extractor']   
ZTF20aatvdvr                            ['lc_feature_extractor']   
ZTF18acyygbu                 ['dimmers', 'lc_feature_extractor']   
ZTF19aaapkyv   ['dimmers', 'lc_feature_extractor', 'extragala...   

                                                     

In [9]:
%%time

# LC features + Alerce stamp classifier results split up over XXX files
# So merge them
alerce_dfs = []
for start_idx, end_idx in zip(range(0, 490000, 10000), range(10000, 500000, 10000)): #90199, 90299
    alerce_df = pd.read_csv(f'../loci_dbs/alerce_cut/merge_df_gr_alerce_mini_start_idx-end_idx={start_idx}-{end_idx}.csv.gz', compression='gzip')
    alerce_dfs.append(alerce_df)
    
merge_df_gr_alerce = pd.concat(alerce_dfs, axis=0).reset_index(drop=True)
merge_df_gr_alerce.drop_duplicates(subset="ztf_object_id", keep='first', inplace=True)
merge_df_gr_alerce = merge_df_gr_alerce.set_index('ztf_object_id')
merge_df_gr_alerce.sort_values('ztf_object_id', inplace=True)
#g_hosts_df.reset_index(inplace=True, drop=True)
merge_df_gr_alerce

CPU times: user 20.5 s, sys: 3.04 s, total: 23.5 s
Wall time: 23.8 s


,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,anomaly_score,anomaly_mask,anomaly_type,is_corrected,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b,stamp_cls
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaabpe,186400,ANT2020pfozy,82.215575,-2.269285,['lc_feature_extractor'],"['2mass_psc', 'bright_guide_star_cat', 'asassn...",NaN,353,212,ztf_candidate:696375333515015062,...,0.407789,Fail,Inlier,True,NaN,NaN,NaN,205.293889,-19.364840,Periodic-Other
ZTF17aaaabpo,452429,ANT2020pe5zi,74.364734,1.978454,['lc_feature_extractor'],"['allwise', '2mass_psc', 'gaia_dr2', 'bright_g...",NaN,344,155,ztf_candidate:696399340915010033,...,0.409668,Pass,Inlier,True,NaN,NaN,NaN,197.156596,-24.116826,E
ZTF17aaaabrl,423624,ANT2020penfy,71.781933,2.417024,['lc_feature_extractor'],"['gaia_dr2', 'csdr2', 'bright_guide_star_cat',...",NaN,1050,342,ztf_candidate:685433871315015006,...,0.426410,Fail,Inlier,True,NaN,NaN,NaN,195.255608,-26.089277,E
ZTF17aaaabsi,272750,ANT2018gcxz2,30.736198,31.004717,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', 'gaia_dr2...",NaN,699,322,ztf_candidate:601411880815010004,...,0.438099,Fail,Inlier,True,NaN,NaN,NaN,140.512575,-29.414199,E
ZTF17aaaabte,277415,ANT2020etrcs,27.864749,33.756283,['lc_feature_extractor'],"['2mass_psc', 'allwise', 'gaia_dr2', 'csdr2', ...",NaN,1840,851,ztf_candidate:598424864615010019,...,0.487671,Fail,Inlier,True,NaN,NaN,NaN,136.973166,-27.473649,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aadbetu,361011,ANT2020aefyvfi,267.016403,2.535582,['lc_feature_extractor'],"['2mass_psc', 'gaia_dr2', 'allwise']",NaN,189,45,ztf_candidate:1360193720815015000,...,0.386911,Pass,Inlier,True,NaN,NaN,NaN,27.878233,15.242485,Periodic-Other
ZTF22aadbfid,934105,ANT2020anrvy,267.958666,13.865609,['lc_feature_extractor'],"['allwise', '2mass_psc', 'gaia_dr2', 'bright_g...",NaN,457,290,ztf_candidate:542313926215010003,...,0.426590,Fail,Inlier,True,NaN,NaN,NaN,38.982263,19.357593,VS
ZTF22aadbfke,153905,ANT2020apq2m,265.318214,7.804713,['lc_feature_extractor'],"['2mass_psc', 'gaia_dr2']",NaN,288,126,ztf_candidate:550314976015015012,...,0.408052,Fail,Inlier,True,NaN,NaN,NaN,32.004076,19.127555,E


# Cut on Transients

In [10]:
np.unique(merge_df_gr_alerce.stamp_cls)

array(['AGN', 'Blazar', 'CEP', 'CV/Nova', 'DSCT', 'E', 'Error', 'LPV',
       'Not_classified', 'Periodic-Other', 'QSO', 'RRL', 'SLSN', 'SN',
       'SNII', 'SNIa', 'SNIbc', 'VS', 'YSO', 'asteroid', 'bogus'],
      dtype=object)

In [12]:
merge_df_gr_alerce_sn = merge_df_gr_alerce[merge_df_gr_alerce.stamp_cls.str.startswith('S')] # 'SLSN', 'SN', 'SNII', 'SNIa', 'SNIbc',
merge_df_gr_alerce_sn.to_csv(f'../loci_dbs/alerce_cut/merge_df_gr_alerce_sn_{len(merge_df_gr_alerce_sn)}objects.csv.gz', compression='gzip')
merge_df_gr_alerce_sn

,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,anomaly_score,anomaly_mask,anomaly_type,is_corrected,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b,stamp_cls
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaadars,858089,ANT2020setxa,74.008152,4.688007,"['dwarf_nova_outburst', 'high_snr', 'high_ampl...",['bright_guide_star_cat'],NaN,176,50,ztf_candidate:726272534215015034,...,NaN,NaN,TBD,NaN,NaN,NaN,NaN,194.434930,-23.001128,SNIa
ZTF17aaadvfh,845168,ANT2020jk2po,10.123141,40.602475,"['in_m31', 'lc_feature_extractor', 'extragalac...","['RC3', '2mass_xsc', 'gaia_dr2']",NaN,345,25,ztf_candidate:1007342854215010039,...,0.428199,Fail,Inlier,NaN,NaN,NaN,NaN,120.687977,-22.221579,SLSN
ZTF17aaahrni,896108,ANT2020aevgf7q,80.423285,-4.186247,"['high_amplitude_transient_candidate', 'nova_t...",[],NaN,63,16,ztf_candidate:1398444022315015011,...,NaN,NaN,TBD,NaN,NaN,NaN,NaN,206.213250,-21.841968,SNIa
ZTF17aaaivqs,985055,ANT2020jq7w6,127.370917,10.773342,"['high_amplitude_variable_star_candidate', 'lc...","['asassn_variable_catalog_v2_20190802', 'allwi...",NaN,358,292,ztf_candidate:671461312615010000,...,0.593062,Fail,Outlier,False,NaN,NaN,NaN,214.180505,26.612885,SLSN
ZTF17aaaizmo,998136,ANT2020pg5q4,132.155248,10.782338,"['matheson_extreme_vpdf', 'lc_feature_extracto...","['2mass_psc', 'bright_guide_star_cat', 'gaia_d...",NaN,289,97,ztf_candidate:680499561915015002,...,0.446122,Fail,Inlier,True,0.1.0,pct_99.99999_outlier,NaN,216.450964,30.870773,SNII
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aacjbmo,909153,ANT2020euljm,205.509960,-19.933047,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', '2mass_ps...",NaN,132,60,ztf_candidate:738499311115015008,...,0.412108,Fail,Inlier,True,NaN,NaN,NaN,318.854533,41.368904,SLSN
ZTF22aackync,274345,ANT2020aeh34ei,302.025743,-2.811300,['lc_feature_extractor'],"['bright_guide_star_cat', '2mass_psc', 'allwis...",NaN,273,88,ztf_candidate:1357212311615015001,...,0.410854,Fail,TBD,True,NaN,NaN,NaN,39.334932,-18.312848,SNII
ZTF22aacreno,281787,ANT2020edvwq,146.940490,-21.929953,['lc_feature_extractor'],"['milliquas', 'bright_guide_star_cat', 'gaia_d...",NaN,129,95,ztf_candidate:828188395815015007,...,0.426595,Fail,Inlier,True,NaN,NaN,NaN,256.172074,23.831250,SLSN


# Run GHOST results for merge_df_gr_alerce_sn in astroghost_for_ANTARES_loci.ipynb, and crossmatch_TNS.ipynb for getting TNS results

# Now Match PS1 objIDs from GHOST into PS1 PSC, and cut to only retain extended sources (galaxies) at location of the host (not transient)

In [2]:
with open("../loci_dbs/loci.pkl", "rb") as f:
    loci = pickle.load(f)
print("done pickle")

done pickle


In [3]:
loci_df = pd.DataFrame.from_dict(loci)
print("made DF")
#loci_df = pd.concat([loci_df.drop(['properties'], axis=1), loci_df['properties'].apply(pd.Series)], axis=1)

made DF


In [4]:
np.unique(loci_df[loci_df.locus_id.str.startswith('ANT2022')].locus_id)

array(['ANT202224ia', 'ANT202224zq', 'ANT2022255q', ..., 'ANT2022zsqq',
       'ANT2022zt4a', 'ANT2022zyfq'], dtype=object)

In [5]:
loci_df[loci_df.locus_id == 'ANT2022nwxx6g061fov']

,locus_id,ra,dec,htm16,properties,tags,catalogs,watch_list_ids,watch_object_ids


Why not longer ANT IDs like ANT2022nwxx6g061fov?

For example, only getting short IDs like 
array(['ANT202224ia', 'ANT20222v7a', 'ANT20223jga', 'ANT20224bta',
       'ANT20224d5a', 'ANT20224nqq', 'ANT20226h6q', 'ANT20226lkq',
       'ANT20226nga', 'ANT20227czq', 'ANT2022a2lq', 'ANT2022a2nmy',
       'ANT2022a2xqy', 'ANT2022a322o', 'ANT2022a35x6', 'ANT2022a42ym',
       'ANT2022a4cms', 'ANT2022a4ooo', 'ANT2022a4ql2', 'ANT2022a4slq',
       'ANT2022a4wga', 'ANT2022a4zjk', 'ANT2022a55vc', 'ANT2022a5a74',
       'ANT2022a5djy', 'ANT2022a5ekq', 'ANT2022a5m6w', 'ANT2022a7354',
       'ANT2022a73tc', 'ANT2022a74wi', 'ANT2022a76dw', 'ANT2022a76mq',
       'ANT2022a76mw', 'ANT2022a76tc', 'ANT2022a776u', 'ANT2022a77fi',
       'ANT2022a7rc2', 'ANT2022a7s2m', 'ANT2022a7t3q', 'ANT2022a7tbs',
       'ANT2022a7tt6', 'ANT2022a7ty6', 'ANT2022a7ud6', 'ANT2022a7v7m',
       'ANT2022a7wks', 'ANT2022a7wky', 'ANT2022a7wmq', 'ANT2022a7wu6',
       'ANT2022a7wye', 'ANT2022a7z5q', 'ANT2022a7zzo', 'ANT2022ae25a',
       'ANT2022ae27y', 'ANT2022ae2am', 'ANT2022ae2jy', 'ANT2022ae2la',
       'ANT2022ae2nq', 'ANT2022ae2p6', 'ANT2022ae2qq', 'ANT2022ae2ti',
       'ANT2022ae2w6', 'ANT2022ae2zu', 'ANT2022ae32g', 'ANT2022ae32m',
       'ANT2022ae334', 'ANT2022ae35q', 'ANT2022ae3ay', 'ANT2022ae3gc',
       'ANT2022ae3js', 'ANT2022ae3jy', 'ANT2022ae3kc', 'ANT2022ae3n2',
       'ANT2022ae3no', 'ANT2022ae3po', 'ANT2022ae3q4', 'ANT2022ae3qo',
       'ANT2022ae3ri', 'ANT2022ae3to', 'ANT2022ae3us', 'ANT2022ae3uw',
       'ANT2022ae3wk', 'ANT2022ae47c', 'ANT2022ae4co', 'ANT2022ae4ey',
       'ANT2022ae4hi', 'ANT2022ae4iw', 'ANT2022ae4ki', 'ANT2022ae4kk',
       'ANT2022ae4pa', 'ANT2022ae4uw', 'ANT2022ae4vq', 'ANT2022ae4w2',
       'ANT2022ae4yq', 'ANT2022ae4zo', 'ANT2022ae57s', 'ANT2022ae5c2',
       'ANT2022ae5iu', 'ANT2022ae5kc', 'ANT2022ae5ni', 'ANT2022ae5ui',
       'ANT2022ae62a', 'ANT2022ae62y', 'ANT2022ae6bm', 'ANT2022ae6do',
       'ANT2022ae6fm', 'ANT2022ae6g2', 'ANT2022ae6rs', 'ANT2022ae6u2',
       'ANT2022ae6um', 'ANT2022ae6z2', 'ANT2022ae77u', 'ANT2022ae7iw',
       'ANT2022ae7jw', 'ANT2022ae7pc', 'ANT2022ae7pw', 'ANT2022ae7vo',
       'ANT2022aeb4u', 'ANT2022aec3s', 'ANT2022aecou', 'ANT2022aedow',
       'ANT2022aeg4g', 'ANT2022aeh3c', 'ANT2022aeh3k', 'ANT2022aeimc',
       'ANT2022aekte', 'ANT2022aeogu', 'ANT2022aeqac', 'ANT2022aerko',
       'ANT2022aesji', 'ANT2022aetda', 'ANT2022aeu2q', 'ANT2022aeuxi',
       'ANT2022aeveg', 'ANT2022aevfa', 'ANT2022aex6q', 'ANT2022aeyu2',
       'ANT2022aeyzi', 'ANT2022aez62', 'ANT2022aez66', 'ANT2022aez6s',
       'ANT2022aezes', 'ANT2022aezhm', 'ANT2022aezk4', 'ANT2022af7ca',
       'ANT2022af7lk', 'ANT2022af7xs', 'ANT2022afa3i', 'ANT2022afa54',
       'ANT2022afafi', 'ANT2022afajo', 'ANT2022afan4', 'ANT2022afapm',
       'ANT2022afawo', 'ANT2022afb3k', 'ANT2022afb3s', 'ANT2022afbfk',
       'ANT2022afbh6', 'ANT2022afbha', 'ANT2022afbka', 'ANT2022afbpe',
       'ANT2022afbps', 'ANT2022afbts', 'ANT2022afbvq', 'ANT2022afbyc',
       'ANT2022afcmw', 'ANT2022afdwg', 'ANT2022afdye', 'ANT2022afeja',
       'ANT2022afemq', 'ANT2022afeuq', 'ANT2022affdg', 'ANT2022affua',
       'ANT2022afgje', 'ANT2022afgvc', 'ANT2022afgye', 'ANT2022afhie',
       'ANT2022afhm4', 'ANT2022afhvi', 'ANT2022afj76', 'ANT2022afjb6',
       'ANT2022afjia', 'ANT2022afjm2', 'ANT2022afkdi', 'ANT2022afkqe',
       'ANT2022afkwe', 'ANT2022aflqc', 'ANT2022afm5y', 'ANT2022afphq',
       'ANT2022afqe2', 'ANT2022afql4', 'ANT2022afqy6', 'ANT2022afrse',
       'ANT2022afsc6', 'ANT2022afsmo', 'ANT2022afsqe', 'ANT2022afsra',
       'ANT2022afsuw', 'ANT2022aftak', 'ANT2022afwvq', 'ANT2022afydi',
       'ANT2022afysu', 'ANT2022ag2iu', 'ANT2022ag45c', 'ANT2022ag4es',
       'ANT2022ag4sy', 'ANT2022ag576', 'ANT2022ag5hw', 'ANT2022ag6bw',
       'ANT2022ag6fu', 'ANT2022agakm', 'ANT2022agd7w', 'ANT2022agdc2',
       'ANT2022aggb4', 'ANT2022aggf4', 'ANT2022aggfs', 'ANT2022aghek',
       'ANT2022aghgm', 'ANT2022agldo', 'ANT2022ago56', 'ANT2022agryw',
       'ANT2022agtjk', 'ANT2022aguxe', 'ANT2022agy6e', 'ANT2022agzgs',
       'ANT2022ah2gk', 'ANT2022ah2z2', 'ANT2022ah5ac', 'ANT2022ahasg',
       'ANT2022ahc3i', 'ANT2022ahcsu', 'ANT2022ahez6', 'ANT2022ahggy',
       'ANT2022ahl7o', 'ANT2022ahmd2', 'ANT2022ahmv4', 'ANT2022ahn6o',
       'ANT2022ahng2', 'ANT2022ahnmk', 'ANT2022ahnq4', 'ANT2022ahnrc',
       'ANT2022ahnt2', 'ANT2022ahnya', 'ANT2022ahnzc', 'ANT2022ahofm',
       'ANT2022ahoqg', 'ANT2022ahowm', 'ANT2022ahpa4', 'ANT2022ahpaq',
       'ANT2022ahpiw', 'ANT2022ahpxk', 'ANT2022ahqya', 'ANT2022ahrbm',
       'ANT2022ahrlc', 'ANT2022ahrw4', 'ANT2022ahsdg', 'ANT2022ahtka',
       'ANT2022ahuas', 'ANT2022ahuh6', 'ANT2022ahvvg', 'ANT2022ahw6u',
       'ANT2022ahwo2', 'ANT2022ahyb6', 'ANT2022ai3uq', 'ANT2022ai5pe',
       'ANT2022aitge', 'ANT2022ajbnq', 'ANT2022ajdzy', 'ANT2022ajeg2',
       'ANT2022ajka6', 'ANT2022ak2ge', 'ANT2022ak2ra', 'ANT2022aklrg',
       'ANT2022akswc', 'ANT2022aksym', 'ANT2022akszi', 'ANT2022akt3s',
       'ANT2022akt3y', 'ANT2022aktbm', 'ANT2022akteo', 'ANT2022aktt4',
       'ANT2022aku2o', 'ANT2022akuqi', 'ANT2022akuyu', 'ANT2022akvlm',
       'ANT2022akvm4', 'ANT2022akvvi', 'ANT2022akvwq', 'ANT2022akvws',
       'ANT2022akvwu', 'ANT2022akwli', 'ANT2022akx2w', 'ANT2022akxp6',
       'ANT2022akxss', 'ANT2022akyeu', 'ANT2022akz2y', 'ANT2022akzmw',
       'ANT2022akzsc', 'ANT2022al7qy', 'ANT2022alb5g', 'ANT2022aldhy',
       'ANT2022alewu', 'ANT2022alf2e', 'ANT2022alfmi', 'ANT2022alo3e',
       'ANT2022alqze', 'ANT2022als5q', 'ANT2022aluoa', 'ANT2022aluq4',
       'ANT2022alvog', 'ANT2022am2am', 'ANT2022am2c2', 'ANT2022am2re',
       'ANT2022amcpy', 'ANT2022amiyu', 'ANT2022amjaq', 'ANT2022amk32',
       'ANT2022amkuw', 'ANT2022amkx6', 'ANT2022aml7w', 'ANT2022amlbk',
       'ANT2022amlhg', 'ANT2022amliy', 'ANT2022amlmi', 'ANT2022amlow',
       'ANT2022amlum', 'ANT2022amm3e', 'ANT2022ammmc', 'ANT2022ammog',
       'ANT2022ammua', 'ANT2022ammvk', 'ANT2022amnbk', 'ANT2022amnhw',
       'ANT2022amnvq', 'ANT2022amnyq', 'ANT2022amnz6', 'ANT2022amnzm',
       'ANT2022amocw', 'ANT2022amodg', 'ANT2022amoia', 'ANT2022amoms',
       'ANT2022amou6', 'ANT2022amp7k', 'ANT2022ampbk', 'ANT2022ampja',
       'ANT2022ampjo', 'ANT2022ampq6', 'ANT2022ampqu', 'ANT2022amqdu',
       'ANT2022amqnw', 'ANT2022amsoe', 'ANT2022amup6', 'ANT2022amvcy',
       'ANT2022amvfy', 'ANT2022amvte', 'ANT2022amw4s', 'ANT2022amwl4',
       'ANT2022amxou', 'ANT2022amzr6', 'ANT2022anqq4', 'ANT2022antjw',
       'ANT2022anvju', 'ANT2022ao5p2', 'ANT2022aoabw', 'ANT2022aoaoq',
       'ANT2022aoaug', 'ANT2022aoeo6', 'ANT2022aog4i', 'ANT2022aoh7q',
       'ANT2022aom5y', 'ANT2022aornm', 'ANT2022ap2fo', 'ANT2022ap3pu',
       'ANT2022ap4ps', 'ANT2022apft6', 'ANT2022apg4q', 'ANT2022apgu6',
       'ANT2022aphla', 'ANT2022apies', 'ANT2022apiue', 'ANT2022apjgm',
       'ANT2022apjho', 'ANT2022apki4', 'ANT2022apkpy', 'ANT2022apm42',
       'ANT2022apm4e', 'ANT2022apmjk', 'ANT2022apmks', 'ANT2022apmms',
       'ANT2022apnom', 'ANT2022apnqw', 'ANT2022apo3q', 'ANT2022appig',
       'ANT2022appva', 'ANT2022aprrk', 'ANT2022aprwq', 'ANT2022aprxc',
       'ANT2022apsh6', 'ANT2022apsi6', 'ANT2022apsmc', 'ANT2022aptja',
       'ANT2022apujo', 'ANT2022apuo4', 'ANT2022apw64', 'ANT2022apxx2',
       'ANT2022aq44c', 'ANT2022aqbgg', 'ANT2022aqcey', 'ANT2022aqcfg',
       'ANT2022aqd4a', 'ANT2022aqe5k', 'ANT2022aqfq2', 'ANT2022aqgdy',
       'ANT2022aqgww', 'ANT2022aqhce', 'ANT2022aqhta', 'ANT2022aqhvq',
       'ANT2022aqjgs', 'ANT2022aqjlw', 'ANT2022aqmby', 'ANT2022aqml2',
       'ANT2022aqndk', 'ANT2022aqnwq', 'ANT2022aqo7a', 'ANT2022aqoqi',
       'ANT2022aqrve', 'ANT2022aqtpi', 'ANT2022aqwes', 'ANT2022aqwpg',
       'ANT2022aqwue', 'ANT2022armxq', 'ANT2022arsqa', 'ANT2022arvvg',
       'ANT2022aryb2', 'ANT2022as2ik', 'ANT2022as3em', 'ANT2022as3mo',
       'ANT2022as45k', 'ANT2022as55a', 'ANT2022as6bu', 'ANT2022as6so',
       'ANT2022as7j4', 'ANT2022asb3o', 'ANT2022asd42', 'ANT2022asljg',
       'ANT2022asrl4', 'ANT2022asu76', 'ANT2022asxfm', 'ANT2022asyk4',
       'ANT2022asyl2', 'ANT2022asylq', 'ANT2022async', 'ANT2022asys4',
       'ANT2022asysc', 'ANT2022asyuo', 'ANT2022asyxk', 'ANT2022asz44',
       'ANT2022aszm4', 'ANT2022aszws', 'ANT2022aszzm', 'ANT2022atayg',
       'ANT2022atbyc', 'ANT2022atege', 'ANT2022atv5m', 'ANT2022audps',
       'ANT2022aueiy', 'ANT2022auekc', 'ANT2022auhgy', 'ANT2022aujgi',
       'ANT2022aurwo', 'ANT2022auskc', 'ANT2022av2tm', 'ANT2022av44o',
       'ANT2022av4gm', 'ANT2022av4y4', 'ANT2022av5hw', 'ANT2022av5uc',
       'ANT2022av6hc', 'ANT2022av6qw', 'ANT2022av6vm', 'ANT2022avg3i',
       'ANT2022avgwi', 'ANT2022avmmc', 'ANT2022avmwe', 'ANT2022avuru',
       'ANT2022avw6m', 'ANT2022avw6y', 'ANT2022avwh6', 'ANT2022avwho',
       'ANT2022avwja', 'ANT2022avxzk', 'ANT2022avzw4', 'ANT2022awa46',
       'ANT2022awato', 'ANT2022awcpu', 'ANT2022awjt6', 'ANT2022ay27y',
       'ANT2022ay4no', 'ANT2022aykiy', 'ANT2022aypdo', 'ANT2022ayqog',
       'ANT2022ayt5y', 'ANT2022ayxqg', 'ANT2022azfuu', 'ANT2022azfwi',
       'ANT2022azgce', 'ANT2022azgrs', 'ANT2022azhko', 'ANT2022azna4',
       'ANT2022aznbc', 'ANT2022aznby', 'ANT2022aznts', 'ANT2022azo3c',
       'ANT2022azo6k', 'ANT2022azo7g', 'ANT2022azobo', 'ANT2022azpgg',
       'ANT2022azq2w', 'ANT2022azqm4', 'ANT2022azqxu', 'ANT2022azrsw',
       'ANT2022azs34', 'ANT2022azt56', 'ANT2022aztfe', 'ANT2022aztfo',
       'ANT2022aztk2', 'ANT2022aztni', 'ANT2022aztpg', 'ANT2022aztye',
       'ANT2022azufk', 'ANT2022ba2zc', 'ANT2022ba4qa', 'ANT2022ba67y',
       'ANT2022ba6x6', 'ANT2022ba7lo', 'ANT2022ba7lq', 'ANT2022baasi',
       'ANT2022baatg', 'ANT2022baatk', 'ANT2022baavo', 'ANT2022babrm',
       'ANT2022bad5g', 'ANT2022baemg', 'ANT2022baepm', 'ANT2022baevs',
       'ANT2022baf2q', 'ANT2022bagwc', 'ANT2022bagyy', 'ANT2022bahyw',
       'ANT2022bai72', 'ANT2022baifk', 'ANT2022baifq', 'ANT2022bain4',
       'ANT2022baj2i', 'ANT2022bajwc', 'ANT2022bajzi', 'ANT2022bak54',
       'ANT2022bakrc', 'ANT2022bal3o', 'ANT2022baluo', 'ANT2022balzk',
       'ANT2022bamhs', 'ANT2022bamn4', 'ANT2022bamtg', 'ANT2022bao7a',
       'ANT2022baoac', 'ANT2022baobc', 'ANT2022bap5i', 'ANT2022bapdk',
       'ANT2022bapo6', 'ANT2022bb3gq', 'ANT2022bbbzw', 'ANT2022bbfyw',
       'ANT2022bbgeo', 'ANT2022bc5gk', 'ANT2022bcgyy', 'ANT2022bch4y',
       'ANT2022bchxo', 'ANT2022bci6c', 'ANT2022bci6y', 'ANT2022bcinw',
       'ANT2022bcivo', 'ANT2022bciw6', 'ANT2022bcksi', 'ANT2022bcpci',
       'ANT2022bdbyu', 'ANT2022bdtcs', 'ANT2022bep3q', 'ANT2022bg2c2',
       'ANT2022bg2wi', 'ANT2022bg54u', 'ANT2022bgz3c', 'ANT2022bhdyy',
       'ANT2022bhfee', 'ANT2022bhgdo', 'ANT2022bhhcm', 'ANT2022bhhdc',
       'ANT2022bhjvk', 'ANT2022bhk54', 'ANT2022bhkje', 'ANT2022bhkmu',
       'ANT2022bhnsy', 'ANT2022bhnve', 'ANT2022bhoh4', 'ANT2022bhp7s',
       'ANT2022bhtvc', 'ANT2022bhvhi', 'ANT2022bic5s', 'ANT2022bijvs',
       'ANT2022bj6iu', 'ANT2022bja2a', 'ANT2022bjl6o', 'ANT2022bjyti',
       'ANT2022bk3ge', 'ANT2022bkudo', 'ANT2022bmkxa', 'ANT2022bmmjk',
       'ANT2022bmsig', 'ANT2022bmxxc', 'ANT2022bq5j2', 'ANT2022bqqa2',
       'ANT2022bqzvu', 'ANT2022brcmw', 'ANT2022brhbc', 'ANT2022brn3o',
       'ANT2022brngs', 'ANT2022brorw', 'ANT2022broxq', 'ANT2022brqku',
       'ANT2022brtos', 'ANT2022brtpk', 'ANT2022bru2a', 'ANT2022bruaq',
       'ANT2022bruso', 'ANT2022bruvy', 'ANT2022bt5rc', 'ANT2022bt6ns',
       'ANT2022buah4', 'ANT2022buare', 'ANT2022buhja', 'ANT2022buhsu',
       'ANT2022bun6c', 'ANT2022buq3k', 'ANT2022buqzy', 'ANT2022burf2',
       'ANT2022burh6', 'ANT2022buxga', 'ANT2022buyge', 'ANT2022bv5uo',
       'ANT2022bvhas', 'ANT2022bvrdo', 'ANT2022bvrmk', 'ANT2022bvt46',
       'ANT2022bvvbq', 'ANT2022dhbq', 'ANT2022ebia', 'ANT2022ehoa',
       'ANT2022euhq', 'ANT2022f37a', 'ANT2022f3mq', 'ANT2022f3va',
       'ANT2022f4ha', 'ANT2022fopa', 'ANT2022fwha', 'ANT2022fy5q',
       'ANT2022gbqa', 'ANT2022gecq', 'ANT2022gezq', 'ANT2022ggaa',
       'ANT2022ggpa', 'ANT2022gjna', 'ANT2022gnga', 'ANT2022gpta',
       'ANT2022gu3a', 'ANT2022hcrq', 'ANT2022k7ca', 'ANT2022kmsq',
       'ANT2022knvq', 'ANT2022kqia', 'ANT2022ksyq', 'ANT2022l36q',
       'ANT2022l6va', 'ANT2022lcaq', 'ANT2022leva', 'ANT2022lfnq',
       'ANT2022lgia', 'ANT2022lhaq', 'ANT2022liia', 'ANT2022ll7a',
       'ANT2022lokq', 'ANT2022lqla', 'ANT2022lrpq', 'ANT2022lsga',
       'ANT2022lyia', 'ANT2022m2cq', 'ANT2022m4ua', 'ANT2022m5ia',
       'ANT2022m5na', 'ANT2022m7aa', 'ANT2022medq', 'ANT2022mfsa',
       'ANT2022mk3q', 'ANT2022moja', 'ANT2022molq', 'ANT2022movq',
       'ANT2022moxq', 'ANT2022mpxa', 'ANT2022mq6q', 'ANT2022mqwa',
       'ANT2022mrca', 'ANT2022mrhq', 'ANT2022mriq', 'ANT2022mrxa',
       'ANT2022msvq', 'ANT2022mtfa', 'ANT2022mtyq', 'ANT2022muxq',
       'ANT2022mvaq', 'ANT2022mxwa', 'ANT2022mybq', 'ANT2022n23a',
       'ANT2022n34a', 'ANT2022n3gq', 'ANT2022n3ja', 'ANT2022n3uq',
       'ANT2022n3vq', 'ANT2022n4oa', 'ANT2022n5aq', 'ANT2022n7ra',
       'ANT2022na4a', 'ANT2022nacq', 'ANT2022naea', 'ANT2022nala',
       'ANT2022nbaa', 'ANT2022ndgq', 'ANT2022ndia', 'ANT2022ndma',
       'ANT2022nf6a', 'ANT2022nfcq', 'ANT2022nfea', 'ANT2022nffq',
       'ANT2022nglq', 'ANT2022ngna', 'ANT2022nhfq', 'ANT2022nhta',
       'ANT2022nj7q', 'ANT2022njaa', 'ANT2022njha', 'ANT2022njlq',
       'ANT2022njyq', 'ANT2022nkra', 'ANT2022nl7a', 'ANT2022nm2a',
       'ANT2022nmjq', 'ANT2022nmsq', 'ANT2022nmzq', 'ANT2022nndq',
       'ANT2022no5q', 'ANT2022npha', 'ANT2022npzq', 'ANT2022nq2q',
       'ANT2022nqbq', 'ANT2022nqea', 'ANT2022nqma', 'ANT2022nqna',
       'ANT2022nqoa', 'ANT2022nqwa', 'ANT2022nqzq', 'ANT2022nrvq',
       'ANT2022nsba', 'ANT2022nt6a', 'ANT2022nteq', 'ANT2022ntfq',
       'ANT2022ntjq', 'ANT2022ntkq', 'ANT2022nu2q', 'ANT2022nv2q',
       'ANT2022nvpa', 'ANT2022nvya', 'ANT2022nwbq', 'ANT2022nwoq',
       'ANT2022nwta', 'ANT2022nx2a', 'ANT2022nx5a', 'ANT2022nxia',
       'ANT2022nxsq', 'ANT2022nyhq', 'ANT2022nzkq', 'ANT2022o2cq',
       'ANT2022o2da', 'ANT2022o2gq', 'ANT2022o2nq', 'ANT2022o2wa',
       'ANT2022o3ra', 'ANT2022o3ua', 'ANT2022o4va', 'ANT2022o5kq',
       'ANT2022o5tq', 'ANT2022o6rq', 'ANT2022o7fa', 'ANT2022o7ga',
       'ANT2022ob3a', 'ANT2022obra', 'ANT2022obya', 'ANT2022ocaa',
       'ANT2022ocmq', 'ANT2022ocva', 'ANT2022ocza', 'ANT2022odpq',
       'ANT2022oe6a', 'ANT2022of3a', 'ANT2022og2q', 'ANT2022ogkq',
       'ANT2022oh3q', 'ANT2022oi2a', 'ANT2022ojla', 'ANT2022ojya',
       'ANT2022olsa', 'ANT2022olta', 'ANT2022onba', 'ANT2022ongq',
       'ANT2022opba', 'ANT2022opea', 'ANT2022opjq', 'ANT2022opva',
       'ANT2022oqva', 'ANT2022orbq', 'ANT2022oswq', 'ANT2022otea',
       'ANT2022oviq', 'ANT2022oy4q', 'ANT2022oyqa', 'ANT2022p4cq',
       'ANT2022p4dq', 'ANT2022p4ga', 'ANT2022p6aq', 'ANT2022p7fa',
       'ANT2022pana', 'ANT2022patq', 'ANT2022pbtq', 'ANT2022pcia',
       'ANT2022pcsq', 'ANT2022pd7q', 'ANT2022pdaq', 'ANT2022pdoa',
       'ANT2022peea', 'ANT2022phga', 'ANT2022pima', 'ANT2022pjya',
       'ANT2022plha', 'ANT2022pmfq', 'ANT2022pmjq', 'ANT2022pp6q',
       'ANT2022pq2q', 'ANT2022prmq', 'ANT2022pwjq', 'ANT2022q47q',
       'ANT2022q6nq', 'ANT2022qc6a', 'ANT2022qf7q', 'ANT2022qg7a',
       'ANT2022qjaq', 'ANT2022ql4a', 'ANT2022qxnq', 'ANT2022qzmq',
       'ANT2022r3ra', 'ANT2022r43q', 'ANT2022r7za', 'ANT2022rbxq',
       'ANT2022rerq', 'ANT2022rksa', 'ANT2022rmga', 'ANT2022ro2a',
       'ANT2022ro6q', 'ANT2022rp3q', 'ANT2022rqna', 'ANT2022rufa',
       'ANT2022rvla', 'ANT2022ryka', 'ANT2022sbdq', 'ANT2022sdeq',
       'ANT2022se4a', 'ANT2022skia', 'ANT2022stka', 'ANT2022sunq',
       'ANT2022tb3a', 'ANT2022tb3q', 'ANT2022tqqa', 'ANT2022tuwq',
       'ANT2022tzhq', 'ANT2022unyq', 'ANT2022up3a', 'ANT2022v3na',
       'ANT2022v7fq', 'ANT2022vigq', 'ANT2022vj4a', 'ANT2022vsaa',
       'ANT2022wrmq', 'ANT2022xjma', 'ANT2022xqlq', 'ANT2022xruq',
       'ANT2022y3qa', 'ANT2022ybaq', 'ANT2022ykfq', 'ANT2022ywmq',
       'ANT2022z5sa', 'ANT2022zcja', 'ANT2022zo4q', 'ANT2022zqjq',
       'ANT2022zt4a'], dtype=object)